<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0112quicktour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- date: 2022_0112
- source url: https://colab.research.google.com/github/huggingface/notebooks/blob/master/transformers_doc/quicktour.ipynb
- filename: 2022_0112quicktour.ipynb


In [ ]:
import platform

isColab = platform.system() == 'Linux'
if isColab:
    # Transformers installation
    !pip install transformers datasets > /dev/null 2>&1
    # To install from source instead of the last release, comment the command above and uncomment the following one.
    #! pip install git+https://github.com/huggingface/transformers.git

In [ ]:
if isColab:
    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                     shell=True).communicate()[0]).decode('utf-8')

    !pip install 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download > /dev/null 2>&1
    !pip install ipadic > /dev/null 2>&1    

# クィックツアー
<!-- # Quick tour -->

それでは「🤗 Transformers」ライブラリの機能を簡単にご紹介します。
このライブラリは，テキストのセンチメントを分析するような自然言語理解 (NLU) 課題や，新しいテキストでプロンプトを完成させたり，別の言語で翻訳したりするような自然言語生成(NLG) 課題のために，事前学習済モデルをダウンロードします。
<!-- Let's have a quick look at the 🤗 Transformers library features. 
The library downloads pretrained models for Natural Language Understanding (NLU) tasks, such as analyzing the sentiment of a text, and Natural Language Generation (NLG), such as completing a prompt with new text or translating in another language.-->

まず，パイプライン API を利用して，事前学習済モデルを推論に利用する方法を説明します。
次に，ライブラリがどのようにしてモデルにアクセスし，データの前処理を行うのかを見ていきます。
<!--First we will see how to easily leverage the pipeline API to quickly use those pretrained models at inference. 
Then, we will dig a little bit more and see how the library gives you access to those models and helps you preprocess your data. -->

<Tip>

ドキュメントで紹介しているすべてのコード例では，左上に Pytorch と TensorFlow のスイッチがあります。
そうでない場合は，コードは何の変更も必要とせずに両方のバックエンドで動作することが期待されます。
<!-- All code examples presented in the documentation have a switch on the top left for Pytorch versus TensorFlow. 
If not, the code is expected to work for both backends without any change needed. -->

</Tip>

## パイプラインを使って課題を開始する
<!-- ## Getting started on a task with a pipeline -->

与えられた課題に訓練済モデルを使用する最も簡単な方法は [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) を使用することです。
<!-- The easiest way to use a pretrained model on a given task is to use [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline). -->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/tiZFewofSLM?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

🤗 Transformer は，以下のような作業をすぐに行うことができます。
<!-- 🤗 Transformers provides the following tasks out of the box:-->

- センチメント分析：テキストがポジティブかネガティブか？
- テキスト生成 (英語)：プロンプトを与えると，モデルがそれに応じるような文を生成する
- 名前実体認識 (NER)：入力文中の各単語に，それが表す実体 (人，場所など) をラベル付けする
- 質問応答：いくつかの文脈と質問をモデルに与え，文脈から答えを抽出する
- マスキングされたテキストを埋める：マスキングされた単 語 (例：`[MASK]`に置き換えられたもの) を含むテキストが与えられた場合，その空白を埋める
- 要約：長いテキストの要約を生成する
- 翻訳：テキストを他の言語に翻訳する
- 特徴抽出：テキストのテンソル表現を返す


<!-- - Sentiment analysis: is a text positive or negative?
- Text generation (in English): provide a prompt and the model will generate what follows.
- Name entity recognition (NER): in an input sentence, label each word with the entity it represents (person, place, etc.)
- Question answering: provide the model with some context and a question, extract the answer from the context.
- Filling masked text: given a text with masked words (e.g., replaced by `[MASK]`), fill the blanks.
- Summarization: generate a summary of a long text.
- Translation: translate a text in another language.
- Feature extraction: return a tensor representation of the text. -->

これが感情分析のためにどのように機能するか見てみましょう (他の課題はすべて [課題の要約](https://huggingface.co/docs/transformers/master/en/task_summary) でカバーされています)。
<!-- Let's see how this work for sentiment analysis (the other tasks are all covered in the [task summary](https://huggingface.co/docs/transformers/master/en/task_summary)): -->

以下の依存関係をインストールしてください (未インストールの場合)。
<!-- Install the following dependencies (if not already installed): -->

```bash
pip install torch
```

```bash
pip install tensorflow
```

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

classifier_ja = pipeline("sentiment-analysis", model="daigo/bert-base-japanese-sentiment",tokenizer="daigo/bert-base-japanese-sentiment")


このコマンドを初めて入力すると，事前学習されたモデルとそのトークン化器がダウンロードされ，キャッシュされます。
トークン化器の役割は，モデルのためにテキストを前処理することで，モデルは予測を行うことになります。
パイプラインでは，これらすべてをグループ化し，予測結果を読めるように後処理します。
例えば；
<!-- When typing this command for the first time, a pretrained model and its tokenizer are downloaded and cached. 
We will look at both later on, but as an introduction the tokenizer's job is to preprocess the text for the model, which is then responsible for making predictions. 
The pipeline groups all of that together, and post-process the predictions to make them readable. 
For instance: -->

In [ ]:
print(classifier("We are very happy to show you the 🤗 Transformers library."))

print(classifier_ja('そこはかとなく書き綴れば，怪しうこそ物狂おしけれ。'))

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]
[{'label': 'ネガティブ', 'score': 0.8490612506866455}]


心強い限りですね。
文章のリストに使用することができ，その文章は前処理された後，モデルに供給され、次のような辞書のリストが返されます。
<!-- That's encouraging! 
You can use it on a list of sentences, which will be preprocessed then fed to the model, returning a list of dictionaries like this one: -->

In [ ]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")


results_ja = classifier_ja(["私は腐女子が好きです。", "陰キャの特徴でしょうねぇー", "ねぇムーミン，こっち向いて"])
for result in results_ja:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")    

大規模なデータセットで使用するには [iterating over a pipeline](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines) を参照してください。
<!-- To use with a large dataset, look at [iterating over a pipeline](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines)-->

2 つ目の文が陰性(ネガティブ)に分類されていますが (陽性か陰性かである必要があります)，その得点はかなり中立的です。
<!-- You can see the second sentence has been classified as negative (it needs to be positive or negative) but its score is fairly neutral. -->

デフォルトでは ，このパイプラインのためにダウンロードされたモデルは  `distilbert-base-uncased-finetuned-st-2-english` と呼ばれています。
このモデルについての詳しい情報は [モデルページ](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) をご覧ください。
[DistilBERT アーキテクチャ](https://huggingface.co/docs/transformers/master/en/model_doc/distilbert) を使用しており，SST-2 というデータセットで感情分析課題のために微調整されています。
<!-- By default, the model downloaded for this pipeline is called "distilbert-base-uncased-finetuned-sst-2-english". 
We can look at its [model page](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) to get more information about it. 
It uses the [DistilBERT architecture](https://huggingface.co/docs/transformers/master/en/model_doc/distilbert) and has been fine-tuned on a dataset called SST-2 for the sentiment analysis task. -->

例えば，フランスのデータで訓練された別のモデルを使いたいとします。
研究機関が多くのデータで事前に学習したモデルを集めた [モデルハブ](https://huggingface.co/models) や，コミュニティモデル (通常，特定のデータセットで大規模モデルを微調整したもの) を検索することができます。
タグ「フランス語」と「テキスト分類」を適用すると `nlptown/bert-base-multilingual-uncased-sentiment` という示唆が得られます。
どのように使用できるか見てみましょう。
<!-- Let's say we want to use another model; for instance, one that has been trained on French data. 
We can search through the [model hub](https://huggingface.co/models) that gathers models pretrained on a lot of data by research labs, but also community models (usually fine-tuned versions of those big models on a specific dataset). 
Applying the tags  "French" and "text-classification" gives back a suggestion "nlptown/bert-base-multilingual-uncased-sentiment". 
Let's see how we can use it. -->

使用するモデルの名前を直接 [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) に渡すことができます。
<!-- You can directly pass the name of the model to use to [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline): -->

In [ ]:
kiritubo = "いずれの御時にか、女御・更衣あまた候い給いける中に、いとやんごとなききわにはあらぬが、すぐれて時めき給うありけり"
print(pipeline("sentiment-analysis",model="daigo/bert-base-japanese-sentiment",tokenizer="daigo/bert-base-japanese-sentiment")(kiritubo))

In [ ]:
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
classifier_ja = pipeline("sentiment-analysis", model="daigo/bert-base-japanese-sentiment",tokenizer="daigo/bert-base-japanese-sentiment")

この分類器は，英語，フランス語だけでなく，オランダ語，ドイツ語，イタリア語，スペイン語のテキストを扱うことができるようになりました。 
(訳注： 日本語も可)
この名前の代わりに，事前学習済みのモデルを保存したローカルフォルダを指定することもできます (下記参照)。
また，モデルオブジェクトとそれに関連するトークン化器を渡すこともできます。
<!-- This classifier can now deal with texts in English, French, but also Dutch, German, Italian and Spanish! 
You can also replace that name by a local folder where you have saved a pretrained model (see below). 
You can also pass a model object and its associated tokenizer.-->

これには 2 つのクラスが必要になります。
1 つ目は [AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer) で，選んだモデルに関連するトークン化器をダウンロードして実体化するのに使います。
2 つ目は [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModelForSequenceClassification) (TensorFlowを使用している場合は [TFAutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.TFAutoModelForSequenceClassification)) で， これはモデル自体をダウンロードするために使用します。
他の課題でライブラリを使用していた場合は，モデルのクラスが変わることに注意してください。
チュートリアルの [課題の要約 task summary](https://huggingface.co/docs/transformers/master/en/task_summary) では，どのタスクにどのクラスが使われているかをまとめています。
<!-- We will need two classes for this. 
The first is [AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer), which we will use to download the tokenizer associated to the model we picked and instantiate it. 
The second is [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModelForSequenceClassification) (or [TFAutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.TFAutoModelForSequenceClassification) if you are using TensorFlow), which we will use to download the model itself. 
Note that if we were using the library on an other task, the class of the model would change. 
The [task summary](https://huggingface.co/docs/transformers/master/en/task_summary) tutorial summarizes which class is used for which task. -->

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

さて，先ほど見つけたモデルとトークン化器をダウンロードするには `from_pretrained()` メソッドを使うだけです。
(ご自由にモデルハブから他のモデルに置き換えてください)。
<!-- Now, to download the models and tokenizer we found previously, we just have to use the `from_pretrained()` method (feel free to replace `model_name` by
any other model from the model hub): -->

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


model_name_ja = "daigo/bert-base-japanese-sentiment"
model_ja = AutoModelForSequenceClassification.from_pretrained(model_name_ja)
tknz_ja = AutoTokenizer.from_pretrained(model_name_ja)
classifier_ja = pipeline("sentiment-analysis", model=model_ja, tokenizer=tknz_ja)

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

# このモデルは PyTorch にしか存在しないので _from_pt_ フラグを使って TensorFlowでそのモデルをインポートします。
# This model only exists in PyTorch, so we use the _from_pt_ flag to import that model in TensorFlow.
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

model_name_ja = "daigo/bert-base-japanese-sentiment"
model_ja = TFAutoModelForSequenceClassification.from_pretrained(model_name_ja, from_pt=True)
tknz_ja = AutoTokenizer.from_pretrained(model_name_ja)
classifier_ja = pipeline("sentiment-analysis", model=model_ja, tokenizer=tknz_ja)

自分のデータに似たデータで事前学習されたモデルが見つからない場合は，自分のデータで事前学習されたモデルを微調整する必要があります。
そのための [サンプルスクリプト](https://huggingface.co/docs/transformers/master/en/examples) を用意しています。
調整が終わったら， [このチュートリアル](https://huggingface.co/docs/transformers/master/en/model_sharing) を使って，ハブ上で微調整したモデルをコミュニティと共有することをお忘れなく。
<!-- If you don't find a model that has been pretrained on some data similar to yours, you will need to fine-tune a pretrained model on your data. We provide [example scripts](https://huggingface.co/docs/transformers/master/en/examples) to do so. 
Once you're done, don't forget to share your fine-tuned model on the hub with the community, using [this tutorial](https://huggingface.co/docs/transformers/master/en/model_sharing). -->

<a id='pretrained-model'></a>

## 内部処理：訓練済モデル
<!-- ## Under the hood: pretrained models -->

これらのパイプラインを使ったときに，内部で何が起こっているのかを見てみましょう。
<!-- Let's now see what happens beneath the hood when using those pipelines. -->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/AhChOFRegn4?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

先ほど見たように，モデルとトークン化器は `from_pretrained` メソッドを使って作成されます。
<!-- As we saw, the model and tokenizer are created using the `from_pretrained` method: -->

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model_name_ja = "daigo/bert-base-japanese-sentiment"
pt_model_ja = AutoModelForSequenceClassification.from_pretrained(model_name_ja)
tknz_ja = AutoTokenizer.from_pretrained(model_name_ja)

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#model_name_ja = "daigo/bert-base-japanese-sentiment"
#tf_model_ja = TFAutoModelForSequenceClassification.from_pretrained(model_name_ja)
#tknz_ja = AutoTokenizer.from_pretrained(model_name_ja)

### トークン化器の使用 
<!-- ### Using the tokenizer -->

トークン化器は，テキストの前処理を担当すると述べました。
まず，与えられたテキストを通常 _tokens_ と呼ばれる単語 (または単語の一部，句読点など) に分割します。
この処理には複数の規則があります (詳細は [トークン化器の要約 tokenizer summary](https://huggingface.co/docs/transformers/master/en/tokenizer_summary) をご覧ください)。
そのため，モデルの名前を使ってトークン化器を実体化し，モデルが事前学習されたときと同じ規則を使用する必要があります。

第 2 ステップは，トークンを数値に変換し，それらからテンソルを構築してモデルに供給できるようにすることです。これを行うために，トークン化器は _vocab_ を持っています。
これは `from_pretrained` メソッドで実体化する際にダウンロードする部分で，モデルが事前学習されたときと同じ_vocab_ を使用する必要があるからです。

これらのステップを与えられたテキストに適用するには，トークン化器に与えればよいのです。
<!-- We mentioned the tokenizer is responsible for the preprocessing of your texts. 
First, it will split a given text in words (or part of words, punctuation symbols, etc.) usually called _tokens_. 
There are multiple rules that can govern that process (you can learn more about them in the [tokenizer summary](https://huggingface.co/docs/transformers/master/en/tokenizer_summary)), which is why we need to instantiate the tokenizer using the name of the model, to make sure we use the same rules as when the model was pretrained.

The second step is to convert those _tokens_ into numbers, to be able to build a tensor out of them and feed them to the model. To do this, the tokenizer has a _vocab_, which is the part we download when we instantiate it with the `from_pretrained` method, since we need to use the same _vocab_ as when the model was pretrained.

To apply these steps on a given text, we can just feed it to our tokenizer: -->

In [ ]:
inputs = tokenizer("We are very happy to show you the 🤗 Transformers library.")

inputs_ja = tokenizer("山路を登りながら、こう考えた。智に働けば角かどが立つ。情に棹さおさせば流される。意地を通とおせば窮屈だ。とかくに人の世は住みにくい。")

上は，辞書文字列を int リストにして返します。
前述のように [ids of the tokens](https://huggingface.co/docs/transformers/master/en/glossary#input-ids) が含まれていますが，モデルに有用な追加の引数も含まれています。
例えば，ここでは，モデルが系列をより良く理解するために使用する [注意マスク](https://huggingface.co/docs/transformers/master/en/glossary#attention-mask) もあります。
<!-- This returns a dictionary string to list of ints. 
It contains the [ids of the tokens](https://huggingface.co/docs/transformers/master/en/glossary#input-ids), as mentioned before, but also additional arguments that will be useful to the model. Here for instance, we also have an [attention mask](https://huggingface.co/docs/transformers/master/en/glossary#attention-mask) that the model will use to have a better understanding of the sequence: -->

In [ ]:
print(inputs)
print(inputs_ja)

文章のリストをトークン化器に直接渡すことができます。
文章を一括してモデルに送りたい場合は，すべての文章を同じ長さにして，モデルが受け入れられる最大の長さに切り詰めて，テンソルを返してほしいと思うでしょう。
トークン化器にはこれらすべてを指定することができます。
<!-- You can pass a list of sentences directly to your tokenizer. 
If your goal is to send them through your model as a batch, you probably want to pad them all to the same length, truncate them to the maximum length the model can accept and get tensors back. 
You can specify all of that to the tokenizer: -->

In [ ]:
pt_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
)

pt_batch_ja = tknz_ja(
    ["山路を登りながら、こう考えた。", "智に働けば角かどが立つ。", "情に棹さおさせば流される。", "意地を通とおせば窮屈だ。", "とかくに人の世は住みにくい。"],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
)

In [ ]:
# tf_batch = tokenizer(
#     ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
#     padding=True,
#     truncation=True,
#     max_length=512,
#     return_tensors="tf",
# )

パディングは，モデルが予想する側 (ここでは右側) に，モデルが事前に学習したパディング・トークンを用いて自動的に適用されます。
注意マスクもパディングを考慮して適応されます。
<!-- The padding is automatically applied on the side expected by the model (in this case, on the right), with the padding token the model was pretrained with. 
The attention mask is also adapted to take the padding into account: -->

In [ ]:
for key, value in pt_batch.items():
    print(f"{key}: {value.numpy().tolist()}")

print('-' * 77)

for key, value in pt_batch_ja.items():
    print(f"{key}: {value.numpy().tolist()}")    

In [ ]:
# for key, value in tf_batch.items():
#     print(f"{key}: {value.numpy().tolist()}")

トークン化器については [こちら](https://huggingface.co/docs/transformers/master/en/preprocessing) をご覧ください。
<!-- You can learn more about tokenizers [here](https://huggingface.co/docs/transformers/master/en/preprocessing). -->

### モデルの使用 
<!-- ### Using the model -->

入力がトークン化器によって前処理されたら，それを直接モデルに送ることができます。
前述したように，モデルが必要とするすべての関連情報が含まれます。
TensorFlow モデルを使用している場合は，辞書のキーを直接テンソルに渡すことができますが，PyTorch モデルの場合は，`**` を追加して辞書をアンパックする必要があります。
<!-- Once your input has been preprocessed by the tokenizer, you can send it directly to the model. 
As we mentioned, it will contain all the relevant information the model needs. 
If you're using a TensorFlow model, you can pass the dictionary keys directly to tensors, for a PyTorch model, you need to unpack the dictionary by adding `**`. -->


In [ ]:
pt_outputs = pt_model(**pt_batch)
pt_outputs_ja = pt_model_ja(**pt_batch_ja)

In [ ]:
#tf_outputs = tf_model(tf_batch)

Transformers では，すべての出力は，モデルの最終的な活性化と他のメタデータを含むオブジェクトです。
これらのオブジェクトについては [こちら](https://huggingface.co/docs/transformers/master/en/main_classes/output) で詳しく説明しています。
とりあえず，出力を自分で調べてみましょう。
<!-- In 🤗 Transformers, all outputs are objects that contain the model's final activations along with other metadata. 
These objects are described in greater detail [here](https://huggingface.co/docs/transformers/master/en/main_classes/output). 
For now, let's inspect the output ourselves: -->

In [ ]:
print(pt_outputs)
print(pt_outputs_ja)

In [ ]:
#print(tf_outputs)

出力オブジェクトが `logits` 属性を持っていることに注目してください。
これを使って，モデルの最終的な活性値にアクセスすることができます。
<!-- Notice how the output object has a `logits` attribute. You can use this to access the model's final activations. -->

<Tip>

すべての🤗 Transformers モデル (PyTorch または TensorFlow) は，最終的な活性化関数 (SoftMax のような) の前に，モデルの活性化を返します。
<!-- All 🤗 Transformers models (PyTorch or TensorFlow) return the activations of the model *before* the final activation function (like SoftMax) since this final activation function is often fused with the loss. -->

</Tip>

予測値を得るために SoftMax の活性値を適用してみましょう。
<!-- Let's apply the SoftMax activation to get predictions. -->

In [ ]:
from torch import nn

pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
pt_predictions_ja = nn.functional.softmax(pt_outputs_ja.logits, dim=-1)

In [ ]:
# import tensorflow as tf
# tf_predictions = tf.nn.softmax(tf_outputs.logits, axis=-1)

先ほどの数字が出てきましたね。
<!-- We can see we get the numbers from before: -->

In [ ]:
print(pt_predictions)
print(pt_predictions_ja)

In [ ]:
#print(tf_predictions)

入力に加えてラベルをモデルに与えた場合，モデルの出力オブジェクトには `loss` 属性も含まれます。
<!-- If you provide the model with labels in addition to inputs, the model output object will also contain a `loss` attribute: -->

In [ ]:
import torch

pt_outputs = pt_model(**pt_batch, labels=torch.tensor([1, 0]))
print(pt_outputs)

#pt_outputs_ja = pt_model_ja(**pt_batch_ja, labels=torch.tensor([1, 0]))
pt_outputs_ja = pt_model_ja(**pt_batch_ja)
print(pt_outputs_ja)

In [ ]:
# import tensorflow as tf

# tf_outputs = tf_model(tf_batch, labels=tf.constant([1, 0]))
# print(tf_outputs)

モデルは標準的な [torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) または [tf.keras.Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model) なので，通常の訓練ループで使用することができます。
🤗 Transformers では，Keras の `fit()` メソッドを利用するのに対して PyTorch での訓練を支援する [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) クラスも提供しています (分散訓練，混合精度などに対応しています)。
詳しくは [training tutorial](https://huggingface.co/docs/transformers/master/en/training) をご覧ください。
<!-- Models are standard [torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) or [tf.keras.Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model) so you can use them in your usual training loop. 
🤗 Transformers also provides a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) class to help with your training in PyTorch (taking care of things such as distributed training, mixed precision, etc.) whereas you can leverage the `fit()` method in Keras. 
See the [training tutorial](https://huggingface.co/docs/transformers/master/en/training) for more details. -->

<Tip>

Pytorch のモデル出力は特別なデータクラスであり，IDE でその属性を自動補完できるようになっています。
また，タプルや辞書のように振る舞うこともできます (例えば，整数やスライス，文字列でインデックスを付けることができます)。
この場合，設定されていない (`None` 値を持つ) 属性は無視されます。
<!-- Pytorch model outputs are special dataclasses so that you can get autocompletion for their attributes in an IDE. 
They also behave like a tuple or a dictionary (e.g., you can index with an integer, a slice or a string) in which case the attributes not set (that have `None` values) are ignored. -->

</Tip>

モデルの微調整が終わったら，次のようにしてトークン化器を使って保存します。
<!-- Once your model is fine-tuned, you can save it with its tokenizer in the following way: -->

In [ ]:
pt_save_directory = "./pt_save_pretrained"

tokenizer.save_pretrained(pt_save_directory)
pt_model.save_pretrained(pt_save_directory)

pt_save_directory_ja = "./pt_save_pretrained_ja"
tknz_ja.save_pretrained(pt_save_directory_ja)
pt_model_ja.save_pretrained(pt_save_directory_ja)

In [ ]:
!ls -lt pt_save_pretrained_ja

In [ ]:
# tf_save_directory = "./tf_save_pretrained"
# tokenizer.save_pretrained(tf_save_directory)
# tf_model.save_pretrained(tf_save_directory)

そして，モデル名の代わりにディレクトリ名を渡して，`AutoModel.from_pretrained()` メソッドを使ってこのモデルをロードバックすることができます。
Transformers の優れた機能の一つに，PyTorch と TensorFlow を簡単に切り替えられることがあります。
<!-- You can then load this model back using the `AutoModel.from_pretrained()` method by passing the directory name instead of the model name. 
One cool feature of 🤗 Transformers is that you can easily switch between PyTorch and TensorFlow: any model saved as before can be loaded back either in PyTorch or TensorFlow.-->

保存したモデルを他のフレームワークでロードしたい場合は，まずそれがインストールされていることを確認してください。
<!-- If you would like to load your saved model in the other framework, first make sure it is installed: -->

```bash
pip install torch
```

```bash
pip install tensorflow
```

そして，対応する Auto クラスを使って，次のように読み込みます。
<!-- Then, use the corresponding Auto class to load it like this: -->

In [ ]:
from transformers import AutoModel

# tokenizer = AutoTokenizer.from_pretrained(tf_save_directory)
# pt_model = AutoModel.from_pretrained(tf_save_directory, from_tf=True)

# tknz_ja = AutoTokenizer.from_pretrained(tf_save_directory_ja)
# pt_model_ja = AutoModel.from_pretrained(tf_save_directory_ja, from_tf=True)

In [ ]:
from transformers import TFAutoModel

tokenizer = AutoTokenizer.from_pretrained(pt_save_directory)
tf_model = TFAutoModel.from_pretrained(pt_save_directory, from_pt=True)

tknz_ja = AutoTokenizer.from_pretrained(pt_save_directory_ja)
tf_model_ja = TFAutoModel.from_pretrained(pt_save_directory_ja, from_pt=True)

最後に，必要であれば，すべての隠れた状態とすべての注目度の重みを返すようにモデルに求めることもできます。
<!-- Lastly, you can also ask the model to return all hidden states and all attention weights if you need them: -->

In [ ]:
pt_outputs = pt_model(**pt_batch, output_hidden_states=True, output_attentions=True)
all_hidden_states = pt_outputs.hidden_states
all_attentions = pt_outputs.attentions

pt_outputs_ja = pt_model_ja(**pt_batch_ja, output_hidden_states=True, output_attentions=True)
all_hidden_states_ja = pt_outputs_ja.hidden_states
all_attentions_ja = pt_outputs_ja.attentions

In [ ]:
# tf_outputs = tf_model(tf_batch, output_hidden_states=True, output_attentions=True)
# all_hidden_states = tf_outputs.hidden_states
# all_attentions = tf_outputs.attentions

### コードへのアクセス
<!-- ### Accessing the code -->

[AutoModel](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModel) と [AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer) クラスは，事前訓練されたモデルであれば自動的に動作するショートカットに過ぎません。
舞台裏では，ライブラリはアーキテクチャとクラスの組み合わせごとに 1 つのモデルクラスを持っているので，コードにアクセスして必要に応じて調整するのは簡単です。
<!-- The [AutoModel](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModel) and [AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer) classes are just shortcuts that will automatically work with any pretrained model. 
Behind the scenes, the library has one model class per combination of architecture plus class, so the code is easy to access and tweak if you need to.-->

先ほどの例では，モデルは `distilbert-base-uncased-finetuned-st-2-english` と呼ばれていましたが，これは [DistilBERT](https://huggingface.co/docs/transformers/master/en/model_doc/distilbert) アーキテクチャを使用していることを意味しています。
[AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModelForSequenceClassification) (TensorFlow を使用している場合は [TFAutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.TFAutoModelForSequenceClassification)) が使用されたので，自動的に作成されたモデルは [DistilBertForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/distilbert#transformers.DistilBertForSequenceClassification) となります。
そのモデルに関連する詳細はドキュメントを見るか，ソースコードを参照してください。
このように，自動生成機能を使わずに，モデルとトークン化器を直接実体化することができます。
<!--In our previous example, the model was called "distilbert-base-uncased-finetuned-sst-2-english", which means it's using the [DistilBERT](https://huggingface.co/docs/transformers/master/en/model_doc/distilbert) architecture. 
As [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModelForSequenceClassification) (or [TFAutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.TFAutoModelForSequenceClassification) if you are using TensorFlow) was used, the model automatically created is then a [DistilBertForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/distilbert#transformers.DistilBertForSequenceClassification). 
You can look at its documentation for all details relevant to that specific model, or browse the source code. This is how you would directly instantiate model and tokenizer without the auto magic: -->

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

In [ ]:
# from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

# model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# model = TFDistilBertForSequenceClassification.from_pretrained(model_name)
# tokenizer = DistilBertTokenizer.from_pretrained(model_name)

### モデルのカスタマイズ
<!-- ### Customizing the model -->

モデル自体の構築方法を変更したい場合は、カスタム構成クラスを定義することができます。
各アーキテクチャにはそれぞれ関連する設定があります。
例えば [DistilBertConfig](https://huggingface.co/docs/transformers/master/en/model_doc/distilbert#transformers.DistilBertConfig)で は，DistilBERT の隠れ層次元，ドロップアウト率などのパラメータを指定することができます。 
隠れ層のサイズの変更などのコアな修正を行った場合，事前訓練されたモデルを使用することができなくなり，ゼロから訓練する必要があります。
そして，この設定から直接モデルを実体化します。
<!-- If you want to change how the model itself is built, you can define a custom configuration class. 
Each architecture comes with its own relevant configuration. For example, [DistilBertConfig](https://huggingface.co/docs/transformers/master/en/model_doc/distilbert#transformers.DistilBertConfig) allows you to specify
parameters such as the hidden dimension, dropout rate, etc for DistilBERT. 
If you do core modifications, like changing the hidden size, you won't be able to use a pretrained model anymore and will need to train from scratch. 
You would then instantiate the model directly from this configuration.-->

以下では [from_pretrained()](https://huggingface.co/docs/transformers/master/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.from_pretrained) メソッドを使って ，トークン化器用の事前定義された語彙を読み込みます。
しかし，トークン化器とは異なり，モデルをゼロから初期化したいと考えています。
そのため [DistilBertForSequenceClassification.from_pretrained()](https://huggingface.co/docs/transformers/master/en/main_classes/model#transformers.PreTrainedModel.from_pretrained) メソッドを使う代わりに，設定からモデルをインスタンス化しています。
<!-- Below, we load a predefined vocabulary for a tokenizer with the [from_pretrained()](https://huggingface.co/docs/transformers/master/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.from_pretrained) method. However, unlike the tokenizer, we wish to initialize the model from scratch. Therefore, we instantiate the model from a configuration instead of using the [DistilBertForSequenceClassification.from_pretrained()](https://huggingface.co/docs/transformers/master/en/main_classes/model#transformers.PreTrainedModel.from_pretrained) method. -->

In [ ]:
from transformers import DistilBertConfig, DistilBertTokenizer, DistilBertForSequenceClassification

config = DistilBertConfig(n_heads=8, dim=512, hidden_dim=4 * 512)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification(config)

In [ ]:
from transformers import DistilBertConfig, DistilBertTokenizer, TFDistilBertForSequenceClassification

config = DistilBertConfig(n_heads=8, dim=512, hidden_dim=4 * 512)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = TFDistilBertForSequenceClassification(config)

モデルの頭の部分 (例えば，ラベル数) だけを変えるものについては，体 (ボディ) の部分には事前学習済みのモデルを使うことができます。
例えば 10 種類のラベルに対応する分類器を，事前に学習された体を使って定義してみましょう。
ラベルの数を変更するために，すべてのデフォルト値を持つ新しいコンフィグレーションを作成する代わりに，コンフィグレーションが取る任意の引数を `from_pretrained` メソッドに渡せば，デフォルトのコンフィグレーションを適切に更新してくれます。
<!-- For something that only changes the head of the model (for instance, the number of labels), you can still use a pretrained model for the body. 
For instance, let's define a classifier for 10 different labels using a pretrained body.
Instead of creating a new configuration with all the default values just to change the number of labels, we can instead pass any argument a configuration would take to the `from_pretrained` method and it will update the default configuration appropriately: -->

In [ ]:
from transformers import DistilBertConfig, DistilBertTokenizer, DistilBertForSequenceClassification

model_name = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=10)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

In [ ]:
from transformers import DistilBertConfig, DistilBertTokenizer, TFDistilBertForSequenceClassification

model_name = "distilbert-base-uncased"
model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=10)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)